In [1]:
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.schema import Document
import pandas as pd
import requests
import tempfile
from tqdm import tqdm_notebook

In [2]:
fallos_df = pd.read_csv('data/fallos.csv')
fallos_df.shape

(4339, 6)

In [3]:
all_docs = []

for idx, row in tqdm_notebook(fallos_df.iterrows()):

    response = requests.get(row['DownloadURL'])
    
    with tempfile.NamedTemporaryFile(suffix='.pdf') as tmp_file:
        tmp_file.write(response.content)
        tmp_file.flush()

        # Extraemos los documentos con PyMuPDF
        loader = PyMuPDFLoader(tmp_file.name)
        docs = loader.load()

        # Le agregamos los metadatos personalizados
        for doc in docs:
            doc.metadata.update({
                "Tribunal": row["Tribunal"],
                "Expediente": row["Expediente N°"],
                "Caratula": row["Carátula"],
                "FechaSentencia": row["Fecha de sentencia"],
                "Sala": row["Sala"]
            })

            all_docs.append(doc)

/var/folders/pg/nt5xcr_n6rd6qh7kjj9bnkpr0000gp/T/ipykernel_17118/2591784553.py:3: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for idx, row in tqdm_notebook(fallos_df.iterrows()):


0it [00:00, ?it/s]

In [37]:
import json

with open("./data/documents_fallos.jsonl", "w", encoding="utf-8") as f:
    for doc in all_docs:
        f.write(json.dumps({
            "page_content": doc.page_content,
            "metadata": doc.metadata
        }, ensure_ascii=False) + "\n")
